This notebook presents an example on how to use Pipelines to chain tools using the PySAGA-cmd package.

This example focuses on getting a hidrologically preprocessed DEM. This is done by chaining two SAGA GIS tools:

- Sink Drainage Route Detection
- Sink Removal

In [1]:
from PySAGA_cmd import (
    SAGA,
    get_sample_dem
)

In [2]:
saga = SAGA()
preprocessing = saga / 'ta_preprocessor'
print(preprocessing.execute().stdout)

Path to "saga_cmd" was not provided. Attempting to find it.
saga_cmd found at "saga_cmd".
____________________________

   #####   ##   #####    ##
  ###     ###  ##       ###
   ###   # ## ##  #### # ##
    ### ##### ##    # #####
 ##### #   ##  ##### #   ##
____________________________

SAGA Version: 8.5.1


Library:	Preprocessing
Category:	Terrain Analysis
File:	/usr/local/lib/saga/libta_preprocessor.so
Description:
Tools for the preprocessing of digital terrain models.

Tools:
 [0]	Flat Detection
 [1]	Sink Drainage Route Detection
 [2]	Sink Removal
 [3]	Fill Sinks (Planchon/Darboux, 2001)
 [4]	Fill Sinks (Wang & Liu)
 [5]	Fill Sinks XXL (Wang & Liu)
 [6]	Burn Stream Network into DEM
 [7]	Breach Depressions

type -h or --help for further information




In [9]:
route_detection = preprocessing / 'Sink Drainage Route Detection'
sink_removal = preprocessing / 'Sink Removal'

sample_dem = get_sample_dem()

pipeline = (
    route_detection(elevation=sample_dem, sinkroute='temp.tif')
    | sink_removal(dem=route_detection.elevation, sinkroute=route_detection.sinkroute,
                   dem_preproc='temp.tif')
)

print('Object type:', type(pipeline))
print('Tools:', pipeline.tools)
print('Tool count:', len(pipeline), end='\n'*2)

outputs = pipeline.execute(verbose=True)

Object type: <class 'PySAGA_cmd.saga.Pipeline'>
Tools: [Tool(library=Library(saga=SAGA(saga_cmd=SAGACMD(path=PosixPath('saga_cmd')), version=Version(major=8, minor=5, patch=1)), library='ta_preprocessor'), tool='Sink Drainage Route Detection', parameters={'elevation': '/media/alex/alex/python-modules-packages-utils/scripts/geomorphology/tools/PySAGA_cmd/PySAGA_cmd/assets/DEM_30m.tif', 'sinkroute': '/tmp/tmp7vwnzyj3/sinkroute_1713088461.tif'}), Tool(library=Library(saga=SAGA(saga_cmd=SAGACMD(path=PosixPath('saga_cmd')), version=Version(major=8, minor=5, patch=1)), library='ta_preprocessor'), tool='Sink Removal', parameters={'dem': '/media/alex/alex/python-modules-packages-utils/scripts/geomorphology/tools/PySAGA_cmd/PySAGA_cmd/assets/DEM_30m.tif', 'sinkroute': '/tmp/tmp7vwnzyj3/sinkroute_1713088461.tif', 'dem_preproc': '/tmp/tmp7vwnzyj3/dem_preproc_1713088461.tif'})]
Tool count: 2

-------------------------
ta_preprocessor / Sink Drainage Route Detection
    -ELEVATION=/media/alex/alex/

In [10]:
outputs[-1].rasters

{'dem': Raster(path=PosixPath('/media/alex/alex/python-modules-packages-utils/scripts/geomorphology/tools/PySAGA_cmd/PySAGA_cmd/assets/DEM_30m.tif')),
 'sinkroute': Raster(path=PosixPath('/tmp/tmp7vwnzyj3/sinkroute_1713088461.tif')),
 'dem_preproc': Raster(path=PosixPath('/tmp/tmp7vwnzyj3/dem_preproc_1713088461.tif'))}